In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
stemmer = nltk.SnowballStemmer("english")

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense

In [2]:
data =pd.read_csv("Reviews.csv")

In [3]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
data['Text'].values[0]

'I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.'

In [5]:
data['Sentiment']=pd.cut(data['Score'], bins=[0,2,3,5], labels=['Negative', 'Nuetral', 'Positive'])

In [6]:
def clean(text):
    text=str(text).lower()
     # Remove text within square brackets
    text=re.sub('\[.*?\]','',text)
    # Remove URLs
    text=re.sub('https?://\S+|www\.\S+','',text)
    # Remove HTML tags
    text = re.sub('<.*?>+', '', text)
    #  Removing Punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Replace newline characters with spaces
    text = re.sub('\n', '', text)
    # Remove words containing digits
    text = re.sub('\w*\d\w*', '', text)
     # Tokenize the text and remove stopwords
    text = [word for word in text.split(' ') if word not in stopwords]
    text=" ".join(text)
    # Stem the remaining words
    text = [stemmer.stem(word) for word in text.split(' ')]
    # Join the cleaned and stemmed words back into a string
    text=" ".join(text)
    return text

In [7]:
clean(data['Text'])

'         bought sever vital can          product arriv label jumbo salt          confect around          look secret ingredi          great taffi great price  wid                                                            great sesam chickenthi good     im disappoint flavor     star small give      best treat train     satisfi product advertisednam text length  dtype object'

In [8]:
data_cleaned=pd.DataFrame()
data_cleaned['Text']=data["Text"].apply(clean)
data_cleaned['Score']=data['Sentiment']
data_cleaned

,Text,Score
0,bought sever vital can dog food product found ...,Positive
1,product arriv label jumbo salt peanutsth peanu...,Negative
2,confect around centuri light pillowi citrus g...,Positive
3,look secret ingredi robitussin believ found g...,Negative
4,great taffi great price wide assort yummi taf...,Positive
...,...,...
568449,great sesam chickenthi good better restur eate...,Positive
568450,im disappoint flavor chocol note especi weak m...,Negative
568451,star small give one train session tri train ...,Positive
568452,best treat train reward dog good groom lower ...,Positive


In [10]:
text=data_cleaned['Text'].values

In [11]:
max_features=5000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(text)
# tokenizes the input text data into sequences of integer indices
sequences = tokenizer.texts_to_sequences(text)

In [12]:
x=sequences
x= pad_sequences(x)

In [13]:
x

array([[   0,    0,    0, ...,  817,    7,   38],
       [   0,    0,    0, ..., 2735,    7, 4154],
       [   0,    0,    0, ...,  377, 1636, 1266],
       ...,
       [   0,    0,    0, ...,    1,   19,   13],
       [   0,    0,    0, ...,  733, 1103,   40],
       [   0,    0,    0, ...,  705,  594, 1688]])

In [ ]:
y=data_cleaned['Score']

In [ ]:
y

In [15]:
tokens = []
for sentence in text:
    tokens.extend(sentence.split())

In [16]:
word_count = {}
for token in tokens:
    if token in word_count:
        word_count[token] += 1
    else:
        word_count[token] = 1

In [17]:
y=pd.get_dummies(data_cleaned['Score']).values
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.33, random_state = 42)

In [19]:
vocab_size = len(word_count)

In [24]:
model = Sequential([
    tf.keras.layers.Embedding(vocab_size, output_dim=50),
    tf.keras.layers.LSTM(50),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [25]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [26]:
print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model.fit(X_train, Y_train,epochs = 15,batch_size=100,verbose = 2)

Epoch 1/15


In [14]:
x.shape

(568454, 1682)